In [ ]:
#default_exp unet

In [ ]:
#hide
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd drive/MyDrive/noise2noise

[Errno 2] No such file or directory: 'drive/MyDrive/noise2noise'
/content/drive/MyDrive/noise2noise


In [ ]:
#hide
#run to export library
from nbdev.export import notebook2script; notebook2script()

In [ ]:
#export
import torch
from noise2noise.helpers import *
import torch.nn as nn

# Unet

> Unet models

In [ ]:
#export
#hide
class UnetBlockDown(nn.Module):
    
    def __init__(self, in_channel, out_channel):
        super().__init__()
        self.stack = nn.Sequential(
            nn.Conv2d(in_channel, out_channel, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(out_channel, out_channel, 3, padding=1),
            nn.ReLU(),
        )
    def forward(self, x):
        return self.stack(x)
    
class UnetBlockUp(nn.Module):
    
    def __init__(self, in_channel, out_channel):
        super().__init__()
     
        self.stack = nn.Sequential(
            nn.ConvTranspose2d(in_channel, in_channel, 3, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channel, out_channel, 3, padding=1),
            nn.ReLU()
        )
    def forward(self, x):
        return self.stack(x)

In [ ]:
#export
#hide
class UnetBlockDownBatchNorm(nn.Module):
    
    def __init__(self, in_channel, out_channel):
        super().__init__()
        self.stack = nn.Sequential(
            nn.Conv2d(in_channel, out_channel, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(out_channel),
            nn.Conv2d(out_channel, out_channel, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(out_channel)
        )
    def forward(self, x):
        return self.stack(x)
    
class UnetBlockUpBatchNorm(nn.Module):
    
    def __init__(self, in_channel, out_channel):
        super().__init__()
     
        self.stack = nn.Sequential(
            nn.ConvTranspose2d(in_channel, in_channel, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(in_channel),
            nn.ConvTranspose2d(in_channel, out_channel, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(out_channel)
        )
    def forward(self, x):
        return self.stack(x)

In [ ]:
noisy_imgs_1 , noisy_imgs_2 = load_images()
img = to_float_image(noisy_imgs_1[:1])
img.shape

torch.Size([1, 3, 32, 32])

In [ ]:
block_down = UnetBlockDown(3,64)
block_up = UnetBlockUp(64,3)
down = block_down(img)
down.shape, block_up(down).shape

(torch.Size([1, 64, 32, 32]), torch.Size([1, 3, 32, 32]))

In [ ]:
#export
#hide
class Unet(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.block_down_1 = UnetBlockDown(3,64)
        self.block_down_2 = UnetBlockDown(64,128)
        
        self.pool = nn.MaxPool2d(2, stride=2)
        self.up = nn.Upsample(scale_factor=2, mode='bilinear')
        
        self.bottom = nn.Sequential(
            nn.Conv2d(128, 128, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1),
            nn.ReLU()
        )
        
        self.block_up_1 = UnetBlockUp(128,64)
        self.block_up_2 = UnetBlockUp(64,32)
        self.output_layer = nn.Conv2d(32, 3, 1)
    
    def forward(self, x):
        x_1 = self.block_down_1(x)
        x= self.pool(x_1)
        
        x_2 = self.block_down_2(x)
        x = self.pool(x_2)
        
        x = self.bottom(x)
        x = self.up(x)+x_2
        x = self.block_up_1(x)
        
        x = self.up(x)+x_1
        x = self.block_up_2(x)
        x = self.output_layer(x)
        
        return x

In [ ]:
unet = Unet()
unet(img).shape

torch.Size([1, 3, 32, 32])

In [ ]:
#export
#hide
class UnetBatchNorm(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.block_down_1 = UnetBlockDownBatchNorm(3,64)
        self.block_down_2 = UnetBlockDownBatchNorm(64,128)
        
        self.pool = nn.MaxPool2d(2, stride=2)
        self.up = nn.Upsample(scale_factor=2, mode='bilinear')
        
        self.bottom = nn.Sequential(
            nn.Conv2d(128, 128, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(128, 128, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128)
        )
        
        self.block_up_1 = UnetBlockUpBatchNorm(128,64)
        self.block_up_2 = UnetBlockUpBatchNorm(64,32)
        self.output_layer = nn.Conv2d(32, 3, 1)
    
    def forward(self, x):
        x_1 = self.block_down_1(x)
        x= self.pool(x_1)
        
        x_2 = self.block_down_2(x)
        x = self.pool(x_2)
        
        x = self.bottom(x)
        x = self.up(x)+x_2
        x = self.block_up_1(x)
        
        x = self.up(x)+x_1
        x = self.block_up_2(x)
        x = self.output_layer(x)
        
        return x

In [ ]:
unet = Unet()
unet(img).shape

torch.Size([1, 3, 32, 32])